In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("bigdata").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [ ]:
df.count()

2643619

In [32]:
#Pull Data Review ID table 

from pyspark.sql.functions import to_date

review_id = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyy-MM-dd').alias("review_date")]).dropDuplicates()

review_id.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100Q6Y4U6PQY2|   39680316|B007VCHSKU|     615106878| 2014-05-09|
|R101K0VXB5BJKG|   34143274|B008BE7ZB4|     545630304| 2015-04-24|
|R101YNCODM37UN|   15173657|B000U8HN5W|     844077196| 2011-04-04|
|R102ISQH3J5MGX|   19807554|B0038RBYKM|     731751355| 2014-11-26|
|R102X63G049EBW|   51158378|B000G0P4I2|     510180147| 2007-08-26|
|R1037YKYCXXGH8|   38944326|B008GP8I00|     845181216| 2015-07-19|
|R103E9M8IDEJVY|   50007243|B001DKUB8M|     348376734| 2011-12-12|
|R103MVW9KTWKTS|    1890121|B000JNMTN4|     967073870| 2014-11-05|
|R103UNZKUZXM54|   11627094|B00825N3CM|     865801380| 2013-12-07|
|R1043S1CMKVSFG|   15074196|B0009YHTDI|     679746946| 2015-03-22|
|R104CN3MULPL4T|   17822780|B000634LT2|     282198819| 2007-09-15|
|R104MQS7BAGFGJ|   20858727|B003BYQ0A6|     852518060| 2014-12

In [ ]:
#Data Review Count 
review_id.count()

2643619

In [ ]:
#Pull products Table 
products_df = df.select(["product_id","product_title"]).dropDuplicates(subset = ["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0764102885|Saint Bernards (B...|
|6041026522|ADOPT ME (I Need ...|
|6041027537|CAUTION Red Color...|
|B00006OAM1|Petstep 560 PetST...|
|B000084E44|Omega Paw Self-Cl...|
|B00008DFTI|GOLDEN CAT COMPAN...|
|B00008DFTP|Purina Pro Plan F...|
|B00008Q378|C.E.T. Flavored T...|
|B00014F7XU|Merrick Thanksgiv...|
|B00014FBRW|Kitty Nest 2 Cat ...|
|B000255O0O|Super Pet Rainbow...|
|B0002563BI|Kent Marine Saltw...|
|B0002563UY|AquaClear Activat...|
|B00025K1HU|  Hikari First Bites|
|B00026068S|Marshall Ferret N...|
|B00028HN7I|Dr. Harvey's Le D...|
|B00028ZLNQ|Lipiderm Gel Cap ...|
|B0002AATJ0|PetSafe Door Medi...|
|B0002APMH4|TetraMarine Marin...|
|B0002APNFA|OSI Spirulina Pel...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Products Table count 
products_df.count()

239341

In [ ]:
#Customer table 
customer_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count").dropDuplicates(subset = ["customer_id"])
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10270641|             1|
|   18365872|             1|
|   16711087|             1|
|   10742726|             2|
|   41169638|             1|
|   43622307|             1|
|   24540309|             2|
|   28258386|             1|
|   35329257|             2|
|   14552054|             1|
|   14529507|             5|
|   45392827|             5|
|   47282953|             1|
|    8201930|             1|
|   20109760|             2|
|   16405801|             4|
|   15056685|            21|
|   20840575|             2|
|   39048303|             1|
|    5596610|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
#Customer table Count 
customer_df.count()

1415190

In [ ]:
#Pull Vine Table 
vine_table = df.select(["review_id","star_rating","helpful_votes","total_votes","vine"]).dropDuplicates(subset = ["review_id"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R101CJ0JBI9J43|          5|            6|          6|   N|
|R101P7GUI5O1WG|          5|            0|          0|   N|
|R102LL91IX39PK|          5|            0|          0|   N|
|R102YVSWWEAP5K|          5|            0|          0|   N|
|R1042WRU1WDUHY|          5|            7|          8|   N|
|R104G7VC1XPVOO|          5|            1|          1|   N|
|R10557FTU9GHY8|          5|            0|          0|   N|
|R105B7Y5ALJ45D|          5|            1|          1|   N|
|R105C41QCCFLIN|          5|            0|          0|   N|
|R105MO44HU33PJ|          5|            0|          0|   N|
|R10685DHJNVEZJ|          2|            0|          0|   N|
|R106BUZ44UVDHE|          4|            0|          0|   N|
|R106K51LXXJPUW|          5|            0|          1|   N|
|R106SFDWB44AB7|          5|            

In [ ]:
#count Vine Table 
vine_table.count()

2643619

In [ ]:
#Download the Postgres driver to connect tothe 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-21 07:25:46--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-12-21 07:25:46 (5.99 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
# Write to RDS
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/bigdata01"
config = {"user":"postgres", "password": "", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id to table in RDS
review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
 # Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:

# Write customer_df to table in RDS
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vines', mode=mode, properties=config)